# Data Pre-Processing for Predicting Myers Briggs Types
### By Arnav Bhakta$^{1}$ and William Yue$^{1}$
#### $^{1}$ Phillips Academy Andover

##### The authors would like to thank Patrick Chen, Michael Huang, Ali Cy, and William H. Yue for their helpful input and advice in crafting this notebook.

In this notebook, we will be going through the data pre-processing steps that are necessary in order predict Myers Briggs Type Indicators (MBTI). To give a bit of an overview, MBTI is an "introspective self-report questionnaire indicating differing psychological preferences in how people perceive the world and make decisions" [[1]](https://en.wikipedia.org/wiki/Myers%E2%80%93Briggs_Type_Indicator). It "divides everyone into 16 distinct personality types across 4 axis:

* Introversion (I) – Extroversion (E)
* Intuition (N) – Sensing (S)
* Thinking (T) – Feeling (F)
* Judging (J) – Perceiving (P)"

and assigns everyone a label, based on which of the personality types they fulfill [[2]](https://www.kaggle.com/datasnaek/mbti-type). "For example, someone who prefers introversion, intuition, thinking and perceiving would be labelled an INTP in the MBTI system" [[2]](https://www.kaggle.com/datasnaek/mbti-type). In doing so, it is possible to get more of an overview of someone's personality, their preferences, and behaviors, and psychological perspective. Hence, in the current study, we look to levarage machine learning (ML) to correctly and accurately classify people's personalities or MBTIs, based on how it is that they "speak" and interact with others. The data is taken from the Kaggle [(MBTI) Myers-Briggs Personality Type Dataset](https://www.kaggle.com/datasnaek/mbti-type), which provides text of people interacting in a forum, as we presume from reading over samples in the dataset, which are then labeled with their corresponding personality type [[2]](https://www.kaggle.com/datasnaek/mbti-type).

In the current notebook, we have seperated the presented dataset into a smaller set, consistent of two types of labeled data, Introversion (I) and Extroversion (E). Using the provided texts for each of these labels (Introversion being labeled a 1 and Extroversion being labeled a 0), we hope to be able to accurately predict each of these personality types.

In [1]:
print(__doc__)

Automatically created module for IPython interactive environment


### Mounting Google Drive

As data pre-processing was done with Google Colab in Google Drive due to the ease of being able to store files on Drive, we must first mount our Drive, in order to access files on it.

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


### Importing Libraries

We import the below libraries to help us with data pre-processing. Pandas is primarily used for loading in the data from a csv, and creating DataFrames. NumPy is primarily used for creating arrays and simple arithmetic. Regular expression operations (re) are primarily used for splitting up the samples and removing unwanted or implicative text from the dataset. Tokenize and Tokenizer are primarily used to split up sentences into smaller units or words called tokens, to helping to understand the text and build the model, by making it easier to understand the meaning of the text, by analyzing it as a sequence of words. The remaining libraries are used for importing and exporting data.

In [31]:
import pandas as pd
import numpy as np
import re
import tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from bs4 import BeautifulSoup
import requests
import json

### Loading in the Data

Below, we load in the data and read the csv from My Drive, using Pandas, and split up the columns of the dataset into two distinct features: the text and labels. We then cast them to NumPy array from Pandas DataFrames, for ease of use later on.

In [27]:
df = pd.read_csv('/content/gdrive/My Drive/Peitho/0.csv')
df_text = np.array(df['text'])
df_label = np.array(df['label'])

We then create two lists: `df_user` and `personality_types`. `df_user` will hold all of the text in the dataset, after sensitive or hard to understand strings are removed. `personality_types` contains all of the 16 possible personality types one can have, as in the sample text, there are multiple instances of these personality types. So, we go ahead and remove them, as to optimize the training of our model, and ensure that it fully bases its predictions off of more natural language, as the different personality types are not necessarily elements of everday speech

In [28]:
df_user = []
personality_types = ['intj', 'intp', 'entp', 'entj', 'infj', 'infp', 'enfj', 'enfp', 'istj', 'isfj', 'estj', 'esfj', 'istp', 'isfp', 'estp', 'esfp']

In addition to removing all mentions of the different personality types, we also replace all instances of links in the text using the `re.sub()` method, as once again, links do not arise in everyday speech. After removing all such instances of sensitive or hard to understand text from the samples, we for one fix any spacing issues that may have arisen during the removing of these specific types of texts, but also, split the text up into the text of the different users, by splitting the text at all instances of `'|||'`, which serve to indicate a break in the person who is saying the speech. In doing so, we are able to have an individual person's speech matched with their specific label or personality type.

In [2]:
for i in df_text:
  text = i.lower()
  text = re.sub(r'http\S+', '', text)
  text = re.sub(r'https\S+', '', text)
  text = re.sub(r'@[A-Za-z0-9]+','', text)
  for j in personality_types:
    text = re.sub(j, '', text)
  while True:
    before_text = text
    text=text.replace('  ', ' ')
    if before_text == text:
      break
  df_user.append(np.array(text.split('|||')))

Displaying the split text:

In [30]:
df_user = np.array(df_user)
display(df_user)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


array([array(["' and moments sportscenter not top ten plays pranks",
       'what has been the most life-changing experience in your life?',
       ' on repeat for most of today.',
       'may the perc experience immerse you.',
       "the last thing my friend posted on his facebook before committing suicide the next day. rest in peace~ 7. sorry to hear of your distress. it's only natural for a relationship to not be perfection all the time in every moment of existence. try to figure the hard times as times of growth, as...",
       '84389 84390 ...', 'welcome and stuff.', ' game. set. match.',
       "prozac, wellbrutin, at least thirty minutes of moving your legs (and i don't mean moving them while sitting in your same desk chair), weed in moderation (maybe try edibles as a healthier alternative...",
       "basically come up with three items you've determined that each type (or whichever types you want to do) would more than likely use, given each types' cognitive functions and what

### Tokenization

As mentioned above, we tokenize the text, by splitting it up into smaller units, as to optimize our model's ability to analyze the text and discover patterns within it. However, prior to doing so, we first go ahead and flatten the array, by reducing its dimensionality to a 1-dimensional array, in order to be able to tokenize the text, as currently, our text is within a series of nested arrays. We do this, be defining a new list, `df_user_flattened`, and looping through each nested array in the `df_user` array, which currently holds all the texts, and assigning each string of text within these nested arrays as new elements in our new 1-dimensional `df_user_flattened` list.

In [32]:
df_user_flattened = []

In [33]:
for i in df_user:
  for j in i:
    df_user_flattened.append(j)

We then convert `df_user_flattened` to a NumPy array for ease of use, and display its values. As we can see, as opposed to `df_user` which consisted of several arrays nested within each other, `df_user_flattened` contains only 1 array, meaning that we successfuly reduced the dimensionality of our data.

In [34]:
df_user_flattened = np.array(df_user_flattened)
display(df_user_flattened)

array(["' and moments sportscenter not top ten plays pranks",
       'what has been the most life-changing experience in your life?',
       ' on repeat for most of today.', ...,
       'i have seen it, and i agree. i did actually think that the first time i watched the movie, and from the beginning (or when they got their powers) i kinda thought andrew would never work right with...',
       "ok so i have just watched underworld 4 (awakening) and must say it was a really good film, compared to the other films out in the last few months anyway. i don't think it was as good as the first 3...",
       "i would never want to turn off my emotions. sometimes i hide them from the world, but i still need them for me.'"],
      dtype='<U874')

Next we go ahead and tokenize our data. In the current study, we do so using `Tokenizer` from `Keras`. `Tokenizer` takes in a few parameters, when tokenizing the data, which are as follows: num_words returns the ids of the `n` most commonly used words in the dataset, where `n` is the `vocab_size` we defined as `vocab_size = 4000`, `oov_token` is used to replace out of vocabulary words.

In [35]:
vocab_size = 4000 

oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size
                      ,oov_token= oov_tok
                      )
tokenizer.fit_on_texts(df_user_flattened)
word_index = tokenizer.word_index

Post-tokenization, we are able to call `word_index` from `tokenizer`, to get the most commonly used words in our dataset, which are:

In [36]:
word_index

{'<OOV>': 1,
 'i': 2,
 'the': 3,
 'to': 4,
 'a': 5,
 'and': 6,
 'of': 7,
 'you': 8,
 'that': 9,
 'it': 10,
 'is': 11,
 'in': 12,
 'my': 13,
 'but': 14,
 'for': 15,
 'have': 16,
 'with': 17,
 'me': 18,
 'this': 19,
 "i'm": 20,
 'be': 21,
 'not': 22,
 'are': 23,
 'like': 24,
 'on': 25,
 'an': 26,
 'as': 27,
 'so': 28,
 'was': 29,
 'just': 30,
 'if': 31,
 'or': 32,
 'do': 33,
 'what': 34,
 'about': 35,
 "don't": 36,
 'think': 37,
 'people': 38,
 'your': 39,
 'when': 40,
 "it's": 41,
 'at': 42,
 'can': 43,
 'all': 44,
 'know': 45,
 'one': 46,
 'they': 47,
 'more': 48,
 'really': 49,
 'would': 50,
 'we': 51,
 'how': 52,
 'out': 53,
 'because': 54,
 'get': 55,
 'from': 56,
 'am': 57,
 "i've": 58,
 'some': 59,
 'time': 60,
 'he': 61,
 'up': 62,
 'very': 63,
 'there': 64,
 'no': 65,
 'them': 66,
 's': 67,
 'who': 68,
 'feel': 69,
 'much': 70,
 'well': 71,
 'being': 72,
 'too': 73,
 'been': 74,
 'by': 75,
 'love': 76,
 'good': 77,
 'things': 78,
 'other': 79,
 'say': 80,
 'way': 81,
 'something

In [20]:
lens = []
for i in df_user_flattened:
  sample_lens = i.split(" ")
  lens.append(len(sample_lens))

max(lens)

156

### Texts to Sequences

In order to be able to train our model using the available texts, we then use the `texts_to_sequences` method to convert our text to a sequence of integers. This is done by using the most frequent words that we found above, and replacing these words that the tokenizer knows, we integers, such that the model that we will build is able to interpret the text.

In [37]:
tokenized = tokenizer.texts_to_sequences(df_user_flattened)

In [ ]:
tokenized

### Padding

The final step that we take, is padding each of the samples, to be the same length. As seen above, the max number of words in the samples of texts that we are provided with is 156 words, so we pad all of the sequences to a length of 150, to ensure that all of the sequences have the same length. This is done using the `pad_sequences` method, which takes the tokenized and sequenced text that we just defined, and pads all of the sequences or truncates them to a length of 150, by adding on 0s until the sequence has a length of 150, or removing integers from the sequence, until the sequence has a length of 150.

In [39]:
padded = pad_sequences(tokenized, maxlen=150, padding='post', truncating='post')

As seen below, our padding was successful, and leaves us with an array of 405263 sequences of length 150, to pass into our model, so that we can predict MBTIs.

In [41]:
padded.shape

(405263, 150)